In [25]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [26]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\paulc\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [5]:
slide_elem.find('div', class_='content_title')

<div class="content_title">Two of a Space Kind: Apollo 12 and Mars 2020</div>

In [6]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'Two of a Space Kind: Apollo 12 and Mars 2020'

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Apollo 12 and the upcoming Mars 2020 mission may be separated by half a century, but they share several goals unique in the annals of space exploration.'

### Featured Images

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [11]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

### Facts

In [12]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [13]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [89]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [90]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
html = browser.html
hemi_soup = soup(html, 'html.parser')

hemi_items = hemi_soup.find_all('div', class_= "description")

for hemi in hemi_items:
    link = hemi.find("a", href=True)
    href = link['href']
    hemispheres = {"img_url":[], "title":[]}    
    url2 = url + href
#    print(url2)
    browser.visit(url2)
    html2 = browser.html
    hemi_soups = soup(html2, 'html.parser')
    hemi_items2 = hemi_soups.find('div', class_= "container")

    hemi_title = hemi_items2.find('h2', class_='title').text
    link2 = hemi_items2.find('a', target=True)
    hemi_url_rel = link2['href']
    
    
    hemi_url = url + hemi_url_rel
#    print(hemi_url)
#    print(hemi_title)
    hemispheres["img_url"].append(hemi_url)
    hemispheres["title"].append(hemi_title)
    browser.back()
    
    hemisphere_image_urls.append(hemispheres)
    

https://marshemispheres.com/images/full.jpg
Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg
Valles Marineris Hemisphere Enhanced


In [91]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls


[{'img_url': ['https://marshemispheres.com/images/full.jpg'],
  'title': ['Cerberus Hemisphere Enhanced']},
 {'img_url': ['https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'],
  'title': ['Schiaparelli Hemisphere Enhanced']},
 {'img_url': ['https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'],
  'title': ['Syrtis Major Hemisphere Enhanced']},
 {'img_url': ['https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'],
  'title': ['Valles Marineris Hemisphere Enhanced']}]

In [20]:
# 5. Quit the browser
browser.quit()